학습 할 때 : 전부 실행

테스트 할 때 : #for train 빼고 전부 실행

In [7]:
!pip install -U segmentation-models
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-49zc0occ
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-49zc0occ


In [8]:
import cv2
import os
import numpy as np
import glob


In [9]:
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf
import albumentations as A

from PIL import Image
import keras

from sklearn.model_selection import train_test_split


In [10]:
import segmentation_models as sm

Pre-processing

In [114]:
#for train
flist=tf.random.shuffle(glob.glob('/content/drive/MyDrive/data/echocardiography/12800/4800/*.npy'))

In [115]:
test_flist=tf.random.shuffle(glob.glob('/content/drive/MyDrive/data/echocardiography/validation/A*C/*.npy'))

In [21]:
#test a2c/ a4c split test
test_a2c_flist=tf.random.shuffle(glob.glob('/content/drive/MyDrive/data/echocardiography/validation/A2C/*.npy'))
test_a4c_flist=tf.random.shuffle(glob.glob('/content/drive/MyDrive/data/echocardiography/validation/A4C/*.npy'))

In [116]:
#for train


val_flist=flist[:1280]
train_flist=flist[1281:]


In [117]:
def make_test_dataset(test_flist):
  test_flist=tf.data.Dataset.from_tensor_slices(test_flist)
  test_dataset=data_(test_flist)
  test_dataset=test_dataset.map(data_preprocess)
  test_dataset=test_dataset.batch(2)
  test_dataset=test_dataset.prefetch(tf.data.experimental.AUTOTUNE)
  return test_dataset

def make_dataset(train_flist,val_flist):
  train_flist=tf.data.Dataset.from_tensor_slices(train_flist)
  val_flist=tf.data.Dataset.from_tensor_slices(val_flist)
  train_dataset=data_(train_flist)
  val_dataset=data_(val_flist)
  train_dataset=train_dataset.map(data_preprocess)
  val_dataset=val_dataset.map(data_preprocess)
  train_dataset=train_dataset.batch(2)
  val_dataset=val_dataset.batch(2)
  train_dataset=train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
  val_dataset=val_dataset.prefetch(tf.data.experimental.AUTOTUNE)
  return train_dataset,val_dataset

def make_dataset_final(train_flist):
  train_flist=tf.data.Dataset.from_tensor_slices(train_flist)
  
  train_dataset=data_(train_flist)
  
  train_dataset=train_dataset.map(data_preprocess)
  
  train_dataset=train_dataset.batch(2)
  
  train_dataset=train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
  
  return train_dataset

In [118]:
def data_(dataset):

  dataset=dataset.map(file_replace)
  dataset=dataset.map(data_resize)
  
  return  dataset

def file_replace(path):
  
  return tf.strings.regex_replace(path, '.npy', '.png'), path

def data_resize(x,y):

  raw = tf.io.read_file(x)
  img = tf.image.decode_png(raw, channels=3)
  img_np=tf.py_function(data_load,inp=[y],Tout=tf.float32)

  return img,img_np

def data_load(data):
  return np.load(data.numpy())

def data_preprocess(img,label):

  img,label=tf.py_function(data_data,inp=[img,label],Tout=(tf.float32,tf.float32))
  img=tf.cast(img,tf.float32)
  label=tf.cast(label,tf.float32)
  
  return img,label

def data_data(img,label):

  img=cv2.resize(img.numpy(),(608,416),cv2.INTER_AREA)
  img=img/255.0
  label=cv2.resize(label.numpy(),(608,416),cv2.INTER_AREA)
  
  return img,label


  

In [119]:
#for train
train_dataset,val_dataset = make_dataset(train_flist,val_flist)

Training

In [122]:

BACKBONE = 'efficientnetb7'
CLASSES = ['seg']

In [123]:

sm.set_framework('tf.keras')
sm.framework()

'tf.keras'

In [134]:

n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1) 
activation = 'sigmoid' if n_classes == 1 else 'softmax'

model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)


/usr/local/lib/python3.7/dist-packages/keras_applications/imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  str(input_shape[-1]) + ' input channels.')


In [135]:
# jaccard index(jaccard coefficient) & dice coefficient

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection)


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 2.0*(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
  

In [ ]:
#for train
model.load_weights('/content/superbest_model_after_after_LISTT02.h5') 

In [19]:
#LR 0.001
model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss+sm.losses.dice_loss,
    metrics=[sm.metrics.iou_score,jacard_coef,dice_coef]
)

In [136]:
#other lr use
model.compile(
    tf.keras.optimizers.Adam(learning_rate=0.0015),
    loss=sm.losses.bce_jaccard_loss+sm.losses.dice_loss,
    metrics=[sm.metrics.iou_score,jacard_coef,dice_coef]
)

In [137]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('./4ch_{epoch:02d}.h5', verbose=True, save_weights_only=True, mode='auto'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_iou_score', factor=0.66, patience=5, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0),
]


In [ ]:
#for train
history=model.fit(train_dataset,epochs=50,callbacks=callbacks,validation_data=val_dataset)

Epoch 1/50
1159/2560 [============>.................] - ETA: 38:21 - loss: 0.5604 - iou_score: 0.7009 - jacard_coef: 0.7009 - dice_coef: 0.8062

In [ ]:
#all test dataset
test_dataset=make_test_dataset(test_flist)
# A2C/A4C test dataset
test_a2c_dataset=make_test_dataset(test_a2c_flist)
test_a4c_dataset=make_test_dataset(test_a4c_flist)

TEST

In [91]:
model.load_weights('/content/drive/MyDrive/final model/superbest_model_after_after_LISTT02.h5')
his=model.evaluate(test_dataset)

100/100 [==============================] - 8s 77ms/step - loss: 0.1423 - iou_score: 0.9204 - jacard_coef: 0.9204 - dice_coef: 0.9581


In [90]:
his=model.evaluate(test_a2c_dataset)

50/50 [==============================] - 4s 77ms/step - loss: 0.1640 - iou_score: 0.9069 - jacard_coef: 0.9069 - dice_coef: 0.9506


In [92]:
his=model.evaluate(test_a4c_dataset)

50/50 [==============================] - 4s 77ms/step - loss: 0.1118 - iou_score: 0.9365 - jacard_coef: 0.9365 - dice_coef: 0.9671
